# TriMet Vehicle Delays

### A look into vehicle delays in TriMet's rail system. 
<br>
***Kenneth Liao***
<br>
***6/12/2018***

Rail vehicle delays are investigated over a 6 month period from January 1st, 2017 through June 30th, 2017.

Problem Statement:
    1. Which routes have the most delays?
    2. Which routes have the longest delays?
    3. Which stops have the longest delays?

In [1]:
# Import libraries and set default pyplot figure size to 10,10
import os
import pandas as pd
import numpy as np
import pylab
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from config import credentials

# Enable offline plotting
plotly.offline.init_notebook_mode(connected=True)

# Set plotly & mapbox credentials
plotly.tools.set_credentials_file(username=credentials['plotly_user'], 
                                  api_key=credentials['plotly_api_key'])
mapbox_access_token = credentials['mapbox_token']

In [2]:
# Change path to folder containing data
path = 'C:/Users/Kenny/projects/pds/trimet/data/trimet-rail'
os.chdir(path)

In [3]:
# Import the 1-month rail files and append them all into the same table
rail = pd.read_csv('rail-jan.csv')
rail = rail.append(pd.read_csv('rail-feb.csv'))
rail = rail.append(pd.read_csv('rail-mar.csv'))
rail = rail.append(pd.read_csv('rail-apr.csv'))
rail = rail.append(pd.read_csv('rail-may.csv'))
rail = rail.append(pd.read_csv('rail-jun.csv'))

rail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5461424 entries, 0 to 962225
Data columns (total 18 columns):
bearing                       int64
blockID                       int64
delay                         int64
direction                     int64
timestamp                     object
garage                        object
lastLocID                     int64
lastStopSeq                   int64
messageCode                   int64
nextLocID                     int64
nextStopSeq                   int64
routeNumber                   int64
signMessageLong               object
tripID                        int64
type                          object
vehicle_id                    int64
vehicle_location_longitude    float64
vehicle_location_latitude     float64
dtypes: float64(2), int64(12), object(4)
memory usage: 791.7+ MB


In [4]:
rail.head()

,bearing,blockID,delay,direction,timestamp,garage,lastLocID,lastStopSeq,messageCode,nextLocID,nextStopSeq,routeNumber,signMessageLong,tripID,type,vehicle_id,vehicle_location_longitude,vehicle_location_latitude
0,42,9043,-138,0,2017-01-01 00:05:35,ELMO,10572,23,927,10574,24,90,MAX Red Line to Airport,6997146,rail,416,-122.563200,45.560901
1,270,9043,-107,0,2017-01-01 00:07:26,ELMO,10574,24,927,10576,25,90,MAX Red Line to Airport,6997146,rail,416,-122.561480,45.572588
2,299,9043,-138,0,2017-01-01 00:10:39,ELMO,10576,25,927,10579,26,90,MAX Red Line to Airport,6997146,rail,416,-122.571721,45.580453
3,1,9043,-136,0,2017-01-01 00:04:51,ELMO,11616,22,927,10572,23,90,MAX Red Line to Airport,6997146,rail,416,-122.567106,45.556137
4,42,9043,-138,0,2017-01-01 00:05:35,ELMO,10572,23,927,10574,24,90,MAX Red Line to Airport,6997146,rail,417,-122.563200,45.560901


## Data Munging

In [5]:
# Convert timestamp to local datetime
rail['timestamp'] = pd.to_datetime(rail['timestamp'])
rail['timestamp'] = rail['timestamp'].dt.tz_localize('UTC').dt.tz_convert('America/Los_Angeles').dt.tz_localize(None)
rail['hour'] = rail['timestamp'].dt.hour
rail.head()

,bearing,blockID,delay,direction,timestamp,garage,lastLocID,lastStopSeq,messageCode,nextLocID,nextStopSeq,routeNumber,signMessageLong,tripID,type,vehicle_id,vehicle_location_longitude,vehicle_location_latitude,hour
0,42,9043,-138,0,2016-12-31 16:05:35,ELMO,10572,23,927,10574,24,90,MAX Red Line to Airport,6997146,rail,416,-122.563200,45.560901,16
1,270,9043,-107,0,2016-12-31 16:07:26,ELMO,10574,24,927,10576,25,90,MAX Red Line to Airport,6997146,rail,416,-122.561480,45.572588,16
2,299,9043,-138,0,2016-12-31 16:10:39,ELMO,10576,25,927,10579,26,90,MAX Red Line to Airport,6997146,rail,416,-122.571721,45.580453,16
3,1,9043,-136,0,2016-12-31 16:04:51,ELMO,11616,22,927,10572,23,90,MAX Red Line to Airport,6997146,rail,416,-122.567106,45.556137,16
4,42,9043,-138,0,2016-12-31 16:05:35,ELMO,10572,23,927,10574,24,90,MAX Red Line to Airport,6997146,rail,417,-122.563200,45.560901,16


In [6]:
# Use a dictionary to define line colors
message_dict = {}
for message in rail['signMessageLong'].unique():
    if 'Red' in message:
        message_dict[str(message)] = 'Red'
    elif 'Blue' in message:
        message_dict[str(message)] = 'Blue'
    elif 'Green' in message:
        message_dict[str(message)] = 'Green'
    elif 'Orange' in message:
        message_dict[str(message)] = 'Orange'
    elif 'Yellow' in message or 'MAX  To City Center' in message:
        message_dict[str(message)] = 'Yellow'
    else:
        message_dict[str(message)] = 'No Color'
        print(message)
        
message_dict

MAX  to Gresham - MAX to Portland Airport


{'MAX  Red Line to Airport': 'Red',
 'MAX  Red Line to City Center & Beaverton': 'Red',
 'MAX  Blue Line to Gresham': 'Blue',
 'MAX  Blue Line to Ruby Junction/E 197th Ave': 'Blue',
 'MAX  Blue Line to Elmonica/170th': 'Blue',
 'MAX  Blue Line to Hillsboro': 'Blue',
 'MAX  Blue Line to Willow Creek/SW 185th Ave': 'Blue',
 'MAX  Yellow Line to Expo Center': 'Yellow',
 'MAX  Yellow Line to City Center/Milwaukie': 'Yellow',
 'MAX  Yellow Line to Interstate/Rose Quarter': 'Yellow',
 'MAX  Green Line to Clackamas Town Center': 'Green',
 'MAX  Green Line to Gateway TC': 'Green',
 'MAX  Green Line to City Center/PSU': 'Green',
 'MAX  Orange Line to City Center/Expo Center': 'Orange',
 'MAX  Orange Line to City Center/PSU': 'Orange',
 'MAX  Orange Line to Milwaukie': 'Orange',
 'MAX  Blue Line to Gateway TC': 'Blue',
 'MAX  Blue Line to City Center': 'Blue',
 'MAX  Blue Line to Rose Qtr': 'Blue',
 'MAX  To City Center': 'Yellow',
 'MAX  Yellow Line to City Center/PSU': 'Yellow',
 'MAX  Blue Li

In [7]:
rail['lineColor'] = rail['signMessageLong'].map(message_dict)
rail.head()

,bearing,blockID,delay,direction,timestamp,garage,lastLocID,lastStopSeq,messageCode,nextLocID,nextStopSeq,routeNumber,signMessageLong,tripID,type,vehicle_id,vehicle_location_longitude,vehicle_location_latitude,hour,lineColor
0,42,9043,-138,0,2016-12-31 16:05:35,ELMO,10572,23,927,10574,24,90,MAX Red Line to Airport,6997146,rail,416,-122.563200,45.560901,16,Red
1,270,9043,-107,0,2016-12-31 16:07:26,ELMO,10574,24,927,10576,25,90,MAX Red Line to Airport,6997146,rail,416,-122.561480,45.572588,16,Red
2,299,9043,-138,0,2016-12-31 16:10:39,ELMO,10576,25,927,10579,26,90,MAX Red Line to Airport,6997146,rail,416,-122.571721,45.580453,16,Red
3,1,9043,-136,0,2016-12-31 16:04:51,ELMO,11616,22,927,10572,23,90,MAX Red Line to Airport,6997146,rail,416,-122.567106,45.556137,16,Red
4,42,9043,-138,0,2016-12-31 16:05:35,ELMO,10572,23,927,10574,24,90,MAX Red Line to Airport,6997146,rail,417,-122.563200,45.560901,16,Red


In [8]:
rail[['routeNumber', 'lineColor','delay']].groupby(['routeNumber', 'lineColor']).count()

delay
routeNumber lineColor         
90          Red         869139
100         Blue       2442917
            No Color     61687
            Red          23870
190         Orange           8
            Yellow      585471
200         Green       886290
290         Orange      592042

In [9]:
# Remove "No Color" lines
rail_clean = rail.loc[rail['lineColor']!='No Color',:]

rail_clean = rail_clean.loc[(rail_clean['routeNumber']!=100) | (rail_clean['lineColor']!='Red')]

rail_clean = rail_clean.loc[(rail_clean['routeNumber']!=190) | (rail_clean['lineColor']!='Orange'),:]

rail_clean[['routeNumber', 'lineColor','delay']].groupby(['routeNumber', 'lineColor']).count()

,,delay
routeNumber,lineColor,
90,Red,869139
100,Blue,2442917
190,Yellow,585471
200,Green,886290
290,Orange,592042


What's the total number of events by line? This gives us an idea of how busy each line was in the 6-month period.

In [10]:
rail_clean[['lineColor', 'timestamp']].groupby('lineColor').count()

,timestamp
lineColor,
Blue,2442917
Green,886290
Orange,592042
Red,869139
Yellow,585471


## Delay Counts by Route

Now that the data is ready, we can answer the first question:

    1. Which routes have the most delays?

In [11]:
# Take a subset of the data that includes the routeNumber, lineColor, and delay columns,
#and only rows where the delay is negative.
#Perform a groupby on lineColor or routeNumber using count.
#This just counts the total number of timestamps in each category.
delay_count_route = rail_clean[['routeNumber','lineColor','delay']].loc[rail_clean['delay']<0].groupby(['routeNumber', 'lineColor']).count().sort_values('delay', ascending=False).reset_index()
delay_count_route

,routeNumber,lineColor,delay
0,100,Blue,1715426
1,200,Green,629591
2,90,Red,577302
3,290,Orange,459261
4,190,Yellow,437277


In [12]:
# Define RGB for rail line colors
lineColors={'Blue':'rgba(0, 102, 204, 0.8)', 
       'Green':'rgba(51, 204, 51, 0.8)',
       'Red':'rgba(255, 0, 0, 0.8)', 
       'Orange':'rgba(255, 128, 0, 0.8)',
       'Yellow':'rgba(236, 236, 19, 0.8)'}

In [13]:
colors = list(lineColors.values())

data = [go.Bar(x=delay_count_route['lineColor'],
            y=delay_count_route['delay'],
       marker=dict(
       color=colors,
       line=dict(color='rgba(0, 77, 102, .5)',
            width=1,)))]

layout = go.Layout(
    yaxis={'title':'Delay Count', 'titlefont':{'size':16}},
    xaxis={'title':'Rail Line', 'titlefont':{'size':16}},
    showlegend=False,
    margin=dict(t=50)
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='delay-count-route.html')

We can also break the delay count down by signMessageLong.

In [14]:
delay_count_message = rail_clean[['routeNumber', 'lineColor', 'signMessageLong','delay']].loc[rail_clean['delay']<0].groupby(['routeNumber', 'lineColor', 'signMessageLong']).count().sort_values('delay').reset_index()
delay_count_message

,routeNumber,lineColor,signMessageLong,delay
0,190,Yellow,MAX Yellow Line to City Center,2
1,190,Yellow,MAX To City Center,2275
2,100,Blue,MAX Blue Line to Rose Quarter TC,3127
3,100,Blue,MAX Blue Line to Rose Qtr,4132
4,190,Yellow,MAX Yellow Line to City Center/PSU,4895
5,190,Yellow,MAX Yellow Line to Interstate/Rose Quarter,8256
6,100,Blue,MAX Blue Line to City Center,12421
7,90,Red,MAX Red Line to PSU,12445
8,290,Orange,MAX Orange Line to City Center/PSU,13744
9,100,Blue,MAX Blue Line to PSU,14139


In [15]:
colors = []
for row in delay_count_message.index:
    color = delay_count_message.loc[row,'lineColor']
    colors.append(lineColors[str(color)])
colors

data = [go.Bar(x=delay_count_message['delay'],
            y=delay_count_message['signMessageLong'],
               orientation = 'h',
       marker=dict(
       color=colors,
       line=dict(color='rgba(0, 77, 102, .5)',
            width=1,)))]

layout = go.Layout(
    title='Delay Counts by Sign Message',
    yaxis=dict(title='Sign Message', titlefont=dict(size=16),
                  tickfont=dict(size=12),
                  ),
    xaxis={'title':'Delay Count', 'titlefont':{'size':16}},
    showlegend=False,
    margin=dict(pad=5,
    l=350,
    r=100),
    height=700
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='delay-count-signMessage.html')

## Normalized Delay Counts by Routes

In [16]:
norm_delay_message = (rail_clean[['routeNumber', 'lineColor', 'signMessageLong','delay']].loc[rail_clean['delay']<0].groupby(['routeNumber', 'lineColor', 'signMessageLong']).count()/rail_clean[['signMessageLong','delay']].groupby('signMessageLong').count()).sort_values('delay').reset_index()
norm_delay_message

,routeNumber,lineColor,signMessageLong,delay
0,100,Blue,MAX Blue Line to City Center,0.400329
1,100,Blue,MAX Blue Line to Rose Quarter TC,0.431132
2,100,Blue,MAX Blue Line to Rose Qtr,0.440136
3,190,Yellow,MAX Yellow Line to Interstate/Rose Quarter,0.527371
4,100,Blue,MAX Blue Line to Gateway TC,0.556128
5,190,Yellow,MAX Yellow Line to City Center/Milwaukie,0.571136
6,90,Red,MAX Red Line to Airport,0.624559
7,200,Green,MAX Green Line to Rose Quarter TC,0.637458
8,190,Yellow,MAX Yellow Line to City Center/PSU,0.637785
9,100,Blue,MAX Blue Line to City Center and Hillsboro,0.642632


In [17]:
colors = []
for row in norm_delay_message.index:
    color = norm_delay_message.loc[row,'lineColor']
    colors.append(lineColors[str(color)])
colors

data = [go.Bar(x=norm_delay_message['delay'],
            y=norm_delay_message['signMessageLong'],
               orientation = 'h',
       marker=dict(
       color=colors,
       line=dict(color='rgba(0, 77, 102, .5)',
            width=1,)))]

layout = go.Layout(
    title='Percent of Time a Line is Delayed',
    yaxis=dict(title='Sign Message', titlefont=dict(size=16),
                  tickfont=dict(size=12),
                  ),
    xaxis={'title':'Delay Fraction (%)', 'titlefont':{'size':16}},
    showlegend=False,
    margin=dict(pad=5,
    l=350,
    r=100),
    height=700
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='delay-count-signMessage-norm.html')

In [18]:
norm_delay_routes = (rail_clean[['routeNumber','lineColor', 'delay']].loc[rail_clean['delay']<0].groupby(['routeNumber', 'lineColor']).count()/rail_clean[['routeNumber', 'delay']].groupby('routeNumber').count()).sort_values('delay', ascending=False).reset_index()
norm_delay_routes

,routeNumber,lineColor,delay
0,290,Orange,0.775724
1,190,Yellow,0.746881
2,200,Green,0.710367
3,100,Blue,0.702204
4,90,Red,0.664223


In [19]:
colors = []
for row in norm_delay_routes.index:
    color = norm_delay_routes.loc[row,'lineColor']
    colors.append(lineColors[str(color)])
colors

data = [go.Bar(x=norm_delay_routes['lineColor'],
            y=norm_delay_routes['delay']*100,
       marker=dict(
       color=colors,
       line=dict(color='rgba(0, 77, 102, .5)',
            width=1,)))]

layout = go.Layout(
    yaxis={'title':'Delay Fraction (%)', 'titlefont':{'size':16}},
    xaxis={'title':'Rail Line', 'titlefont':{'size':16}},
    showlegend=False,
    margin=dict(t=50)
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='delay-route-norm.html')

## Total delay by Line

In [20]:
idx = rail_clean[['routeNumber',
          'lineColor', 
          'tripID', 
          'vehicle_id', 
          'signMessageLong', 
          'lastStopSeq']
        ].groupby(['routeNumber', 
          'lineColor', 
          'tripID',  
          'vehicle_id', 
          'signMessageLong']
                 )['lastStopSeq'].transform('max') == rail_clean['lastStopSeq']

delay_route_sum = rail_clean.loc[idx, ['lineColor','delay']
        ].loc[rail_clean.loc[idx,'delay']<0].groupby(['lineColor']
                                              ).sum().sort_values('delay').reset_index()
delay_route_sum

,lineColor,delay
0,Blue,-9525584
1,Yellow,-5681365
2,Red,-4118455
3,Orange,-4084661
4,Green,-3040831


In [21]:
delay_route_mean = rail_clean.loc[idx, ['lineColor','delay']
        ].loc[rail_clean.loc[idx,'delay']<0].groupby(['lineColor']
                                              ).mean().sort_values('delay').reset_index()
delay_route_mean

,lineColor,delay
0,Red,-272.204560
1,Blue,-266.137237
2,Green,-231.347459
3,Yellow,-213.200428
4,Orange,-196.993537


In [22]:
# Order the lastLocID starting with the largest total delay.
delay_route_mean['lineColor_cat'] = pd.Categorical(
    delay_route_mean['lineColor'], 
    categories=[val for val in delay_route_sum['lineColor']],
    ordered=True
)
delay_route_mean = delay_route_mean.sort_values('lineColor_cat').reset_index(drop=True).drop('lineColor_cat', axis=1)
delay_route_mean

,lineColor,delay
0,Blue,-266.137237
1,Yellow,-213.200428
2,Red,-272.204560
3,Orange,-196.993537
4,Green,-231.347459


In [23]:
colors = []
for row in delay_route_sum.index:
    color = delay_route_sum.loc[row,'lineColor']
    colors.append(lineColors[str(color)])
colors

trace0 = go.Bar(
    name='Total Delay',
    x=delay_route_sum['lineColor'],
    y=-delay_route_sum['delay']/60/60,
    showlegend=True,
    marker=dict(
        color=colors,
        line=dict(color='rgba(0, 77, 102, .5)',
            width=1,)
    )
)
trace1 = go.Scatter(
    name='Average Delay',
    x=delay_route_mean['lineColor'],
    y=-delay_route_mean['delay'],
    yaxis='y2',
    line = dict(
        color = ('rgb(0, 51, 102)'),
        width = 4)
                    
)

data = [trace0, trace1]
layout = go.Layout(
    yaxis={'title':'Delay (hours)', 'titlefont':{'size':16}},
    xaxis={'title':'Rail Line', 'titlefont':{'size':16}},
    showlegend=True,
    legend=dict(font=dict(size=14),
               orientation='h',
               y=1.1),
    barmode='group',
    yaxis2=dict(title='Delay (seconds)',
                overlaying='y',
                side='right'
    ),
    margin=dict(b=100,
               t=50)
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='delay-route.html')

## Total Delay by StopID

In [24]:
# Check that each stop is associated with only one line color
locID_lineColor = rail_clean[['lineColor', 'lastLocID']].groupby(['lastLocID']).nunique()
locID_lineColor.head()

,lineColor,lastLocID
lastLocID,,
7601,5,1
7606,5,1
7608,5,1
7618,5,1
7627,5,1


In [25]:
tdelay_stopID = rail_clean[['lastLocID', 'delay']].groupby(['lastLocID']).sum().sort_values('delay').reset_index()
tdelay_stopID[0:10]

,lastLocID,delay
0,8347,-13231372
1,8346,-10470551
2,8345,-9590337
3,8344,-8603169
4,8341,-8505411
5,8371,-8489995
6,8343,-8454713
7,8350,-7901631
8,8384,-7876735
9,8342,-7712741


In [26]:
locID_lineColor.loc[locID_lineColor.index.isin(tdelay_stopID.loc[:10,'lastLocID']),:]

,lineColor,lastLocID
lastLocID,,
8341,3,1
8342,3,1
8343,3,1
8344,3,1
8345,3,1
8346,3,1
8347,3,1
8350,1,1
8371,3,1


So the only stop that belongs to a unique lineColor is 8350. Let's see what lines each of the stops belong to.

In [27]:
top10 = rail_clean.loc[rail_clean['lastLocID'].isin(tdelay_stopID.loc[:9,'lastLocID']),['lineColor', 'lastLocID', 'delay']]
top10[:10]

,lineColor,lastLocID,delay
8,Red,8342,58
9,Red,8343,68
10,Red,8344,76
11,Red,8345,64
12,Red,8346,25
17,Red,8342,58
18,Red,8343,68
19,Red,8344,76
20,Red,8345,64
21,Red,8346,25


In [28]:
top10 = top10.groupby(['lastLocID','lineColor']).sum().reset_index()
top10[:10]

,lastLocID,lineColor,delay
0,8341,Blue,-3831808
1,8341,Green,-2364557
2,8341,Red,-2309046
3,8342,Blue,-3548207
4,8342,Green,-2093214
5,8342,Red,-2071320
6,8343,Blue,-3931412
7,8343,Green,-2304109
8,8343,Red,-2219192
9,8344,Blue,-4123368


In [29]:
[val for val in tdelay_stopID.loc[:9,'lastLocID']]

[8347, 8346, 8345, 8344, 8341, 8371, 8343, 8350, 8384, 8342]

In [30]:
# Order the lastLocID starting with the largest total delay.
top10['lastLocID_cat'] = pd.Categorical(
    top10['lastLocID'], 
    categories=[val for val in tdelay_stopID.loc[:9,'lastLocID']],
    ordered=True
)
top10 = top10.sort_values('lastLocID_cat').reset_index(drop=True).drop('lastLocID_cat', axis=1)
top10[:10]

,lastLocID,lineColor,delay
0,8347,Red,-2696563
1,8347,Green,-3786845
2,8347,Blue,-6747964
3,8346,Red,-2638280
4,8346,Green,-2845667
5,8346,Blue,-4986604
6,8345,Green,-2593475
7,8345,Red,-2419728
8,8345,Blue,-4577134
9,8344,Blue,-4123368


In [31]:
# Blue line
trace0 = go.Bar(
    x=top10.loc[top10['lineColor']=='Blue','lastLocID'],
    y=-top10.loc[top10['lineColor']=='Blue','delay']/60/60,
    name='Blue Line',
    marker=dict(
        color=lineColors['Blue'],
        line=dict(color='rgba(0, 77, 102, .5)', width=1)
    )
)
# Green Line
trace1 = go.Bar(
    x=top10.loc[top10['lineColor']=='Green','lastLocID'],
    y=-top10.loc[top10['lineColor']=='Green','delay']/60/60,
    name='Green Line',
    marker=dict(
        color=lineColors['Green'],
        line=dict(color='rgba(0, 77, 102, .5)', width=1)
    )
)
# Red Line
trace2 = go.Bar(
    x=top10.loc[top10['lineColor']=='Red','lastLocID'],
    y=-top10.loc[top10['lineColor']=='Red','delay']/60/60,
    name='Red Line',
    marker=dict(
        color=lineColors['Red'],
        line=dict(color='rgba(0, 77, 102, .5)', width=1)
    )
)
# Total delay by stop ID
trace3 = go.Scatter(
    x=top10.loc[top10['lineColor']=='Blue','lastLocID'],
    y=-tdelay_stopID.loc[0:10,'delay']/60/60,
    yaxis='y2',
    name='Total Delay',
    line = dict(
        color = ('rgb(0, 51, 102)'),
        width = 4)
)

data = [trace0, trace1, trace2, trace3]

# Format the figure
layout = go.Layout(
    yaxis=dict(title='Delay (hours)',
              titlefont=dict(size=16),
              tickfont=dict(size=14)),
    xaxis=dict(title='Stop ID',
              titlefont=dict(size=16),
              type='category',
              tickangle=-45,
              tickfont=dict(size=14)),
    legend=dict(font=dict(size=14),
               orientation='h',
               y=1.1),
    barmode='group',
    yaxis2=dict(title='Delay (hours)',
                overlaying='y',
                side='right'
    ),
    margin=dict(b=100,
               t=50)
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='totalDelay-top10-byLine.html')

In [32]:
# generate the same plot as above but for the top 5 only
top5 = rail_clean.loc[rail_clean['lastLocID'].isin(tdelay_stopID.loc[:4,'lastLocID']),['lineColor', 'lastLocID', 'delay']]
top5 = top5.groupby(['lastLocID','lineColor']).sum().reset_index()

In [33]:
# Order the lastLocID starting with the largest total delay.
top5['lastLocID_cat'] = pd.Categorical(
    top5['lastLocID'], 
    categories=[val for val in tdelay_stopID.loc[:4,'lastLocID']],
    ordered=True
)
top5 = top5.sort_values('lastLocID_cat').reset_index(drop=True).drop('lastLocID_cat', axis=1)
top5[:10]

# generate Plotly graph
trace0 = go.Bar(
    x=top5.loc[top5['lineColor']=='Blue','lastLocID'],
    y=-top5.loc[top5['lineColor']=='Blue','delay']/60/60,
    name='Blue Line',
    marker=dict(
        color=lineColors['Blue'],
        line=dict(color='rgba(0, 77, 102, .5)', width=1)
    )
)
trace1 = go.Bar(
    x=top5.loc[top5['lineColor']=='Green','lastLocID'],
    y=-top5.loc[top5['lineColor']=='Green','delay']/60/60,
    name='Green Line',
    marker=dict(
        color=lineColors['Green'],
        line=dict(color='rgba(0, 77, 102, .5)', width=1)
    )
)
trace2 = go.Bar(
    x=top5.loc[top5['lineColor']=='Red','lastLocID'],
    y=-top5.loc[top5['lineColor']=='Red','delay']/60/60,
    name='Red Line',
    marker=dict(
        color=lineColors['Red'],
        line=dict(color='rgba(0, 77, 102, .5)', width=1)
    )
)
# Total delay by stop ID
trace3 = go.Scatter(
    x=top5.loc[top5['lineColor']=='Blue','lastLocID'],
    y=-tdelay_stopID.loc[0:5,'delay']/60/60,
    yaxis='y2',
    name='Total Delay',
    line = dict(
        color = ('rgb(0, 51, 102)'),
        width = 4)
)

data = [trace0, trace1, trace2, trace3]
layout = go.Layout(
    yaxis=dict(title='Delay (hours)',
              titlefont=dict(size=16)),
    xaxis=dict(title='Stop ID',
              titlefont=dict(size=16),
              type='category',
              tickangle=-45,
              tickfont=dict(size=14)),
    barmode='group',
    yaxis2=dict(title='Delay (hours)',
                overlaying='y',
                side='right'
    ),
    legend=dict(font=dict(size=14),
               orientation='h',
               y=1.1),
    margin=dict(b=100,
               t=50)
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='totalDelay-top5-byLine.html')

Interestingly, the stops with the top 5 longest delays are ones that are shared between the blue, green, and red lines. Also notice that for each stops shown in the plot, the blue line has the largest total delay, followed by the green and red lines. This is consistent across the top 5.

We saw that the stops with the greatest total delay are stops that are shared between the red, blue and green lines. Let's see where these stops are on a map.

### Delays by Stop ID - Maps

I will be visualization stopID data on maps so for that I'll need to get the latitude and longitude coordinates of each location ID.

In [34]:
locations = rail_clean[['lastLocID', 'vehicle_location_longitude', 'vehicle_location_latitude']].groupby('lastLocID').mean()
locations[:10]

,vehicle_location_longitude,vehicle_location_latitude
lastLocID,,
7601,-122.675525,45.526292
7606,-122.683077,45.508800
7608,-122.680410,45.513553
7618,-122.682205,45.510263
7627,-122.676564,45.520613
7646,-122.677978,45.517998
7763,-122.676478,45.527948
7774,-122.682016,45.512795
7777,-122.678573,45.519224


I also need to define the coordinates that I want to center my map plots on.

In [35]:
center_lat = ((locations['vehicle_location_latitude'].max()-locations['vehicle_location_latitude'].min())/2)+locations['vehicle_location_latitude'].min()
center_lon = ((locations['vehicle_location_longitude'].max()-locations['vehicle_location_longitude'].min())/2)+locations['vehicle_location_longitude'].min()

print(center_lat)
print(center_lon)

45.51807743199744
-122.70501593510068


We saw that the stops with the longest delays were all on the red, blue, and green lines. I'll start by visualizing all of the stops for these 3 lines. Then we can plot just the 10 stops and see where they are along these routes.

In [36]:
# get all stop IDs for red, blue, and green lines
rgb = rail_clean.loc[rail_clean['lineColor'].isin(['Blue','Green','Red']),['lineColor', 'lastLocID']].drop_duplicates()
rgb[:10]

,lineColor,lastLocID
0,Red,10572
1,Red,10574
2,Red,10576
3,Red,11616
8,Red,8342
9,Red,8343
10,Red,8344
11,Red,8345
12,Red,8346
26,Red,8333


In [37]:
rail_clean[['lineColor', 'lastLocID']].groupby(['lineColor']).nunique()

,lineColor,lastLocID
lineColor,,
Blue,1,113
Green,1,48
Orange,1,26
Red,1,65
Yellow,1,35


In [38]:
rgb_coords = rgb.set_index('lastLocID').join(locations)
rgb_coords[:10]

,lineColor,vehicle_location_longitude,vehicle_location_latitude
lastLocID,,,
7601,Green,-122.675525,45.526292
7601,Red,-122.675525,45.526292
7601,Blue,-122.675525,45.526292
7606,Green,-122.683077,45.508800
7606,Red,-122.683077,45.508800
7606,Blue,-122.683077,45.508800
7608,Green,-122.680410,45.513553
7608,Red,-122.680410,45.513553
7608,Blue,-122.680410,45.513553


Now we have the coordinates for each stop in the Blue, Green, and red lines. I have to add some small amount of random jitter to each stopID so that points for each color line don't overlap on each other.

In [39]:
jitter = (np.random.rand(len(rgb_coords))-0.5)*.0002
jitter[:20]

array([-9.78878215e-05,  5.96439333e-05, -6.23360374e-05,  1.39097612e-05,
       -4.62148459e-05,  6.91306741e-05,  7.53684433e-05,  3.69873697e-05,
        4.57610759e-05, -4.23926959e-05,  8.80178573e-05, -1.09915932e-05,
        5.93281374e-05, -6.93788133e-05,  8.14867406e-05,  3.20502719e-06,
        6.23279488e-06, -7.74718410e-05,  5.83160143e-05,  8.59157582e-05])

In [40]:
rgb_coords_jitter = rgb_coords.copy()
rgb_coords_jitter['vehicle_location_longitude'] = (rgb_coords_jitter['vehicle_location_longitude']+jitter).round(5)
rgb_coords_jitter['vehicle_location_latitude'] = (rgb_coords_jitter['vehicle_location_latitude']+jitter).round(5)
rgb_coords_jitter = rgb_coords_jitter.reset_index()
rgb_coords_jitter[:10]

,lastLocID,lineColor,vehicle_location_longitude,vehicle_location_latitude
0,7601,Green,-122.67562,45.52619
1,7601,Red,-122.67547,45.52635
2,7601,Blue,-122.67559,45.52623
3,7606,Green,-122.68306,45.50881
4,7606,Red,-122.68312,45.50875
5,7606,Blue,-122.68301,45.50887
6,7608,Green,-122.68033,45.51363
7,7608,Red,-122.68037,45.51359
8,7608,Blue,-122.68036,45.51360
9,7618,Green,-122.68225,45.51022


In [41]:
# Separate each lineColor into its own dataframe
blueLine = rgb_coords_jitter.loc[rgb_coords_jitter['lineColor']=='Blue',:]
greenLine = rgb_coords_jitter.loc[rgb_coords_jitter['lineColor']=='Green',:]
redLine = rgb_coords_jitter.loc[rgb_coords_jitter['lineColor']=='Red',:]

In [42]:
bLocID = list(map(str, blueLine['lastLocID']))
blueText_all = ['Stop ID: ' + s for s in bLocID]
gLocID = list(map(str, greenLine['lastLocID']))
greenText_all = ['Stop ID: ' + s for s in gLocID]
rLocID = list(map(str, redLine['lastLocID']))
redText_all = ['Stop ID: ' + s for s in rLocID]

In [43]:
# blue line
trace0 = go.Scattermapbox(
        name='Blue Line',
        lat=blueLine['vehicle_location_latitude'],
        lon=blueLine['vehicle_location_longitude'],
        mode='markers',
        marker=dict(
            size=10,
            color=lineColors['Blue']
        ),
        text=blueText_all
    )

# green line
trace1 = go.Scattermapbox(
        name='Green Line',
        lat=greenLine['vehicle_location_latitude'],
        lon=greenLine['vehicle_location_longitude'],
        mode='markers',
        marker=dict(
            size=10,
            color=lineColors['Green']
        ),
        text=greenText_all
    )

# red line
trace2 = go.Scattermapbox(
        name='Red Line',
        lat=redLine['vehicle_location_latitude'],
        lon=redLine['vehicle_location_longitude'],
        mode='markers',
        marker=dict(
            size=10,
            color=lineColors['Red']
        ),
        text=redText_all
    )

layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=center_lat-.005,
            lon=center_lon
        ),
        pitch=0,
        zoom=9,
    ),
    margin=dict(l=0,
               r=0,
               t=30,
               b=50),
    legend=dict(font=dict(size=14),
               orientation='h',
               y=1.1)
    
)

fig = dict(data=[trace0, trace1, trace2], layout=layout)
plotly.offline.iplot(fig, filename='red-green-blue-stops.html')

The plot above shows all stop IDs for the blue, green, and red rail lines. A portion of the points overlap between the lines. The small amount of jitter we introduced into the stop coordinates allow us to resolve the overlapping points when zooming in. Hovering over each point displays the stop ID, in addition to the coordinates.

Now I want to plot only the top 5 stops with the longest delays.

In [44]:
# Add the coordinates for each stop
top5_coords = top5.set_index('lastLocID').join(locations).reset_index()
top5_coords[:10]

,lastLocID,lineColor,delay,vehicle_location_longitude,vehicle_location_latitude
0,8341,Blue,-3831808,-122.660870,45.530076
1,8341,Green,-2364557,-122.660870,45.530076
2,8341,Red,-2309046,-122.660870,45.530076
3,8344,Blue,-4123368,-122.619980,45.532580
4,8344,Green,-2317606,-122.619980,45.532580
5,8344,Red,-2162195,-122.619980,45.532580
6,8345,Blue,-4577134,-122.601614,45.529342
7,8345,Green,-2593475,-122.601614,45.529342
8,8345,Red,-2419728,-122.601614,45.529342
9,8346,Blue,-4986604,-122.576508,45.532783


In [45]:
jitter = (np.random.rand(len(top5_coords))-0.5)*.0003
jitter[:20]

array([ 5.99734234e-06, -3.75157133e-05,  6.99311463e-05, -1.03794105e-04,
        9.16897327e-05, -9.51062073e-05, -3.13608782e-05,  6.43284998e-05,
        1.18119867e-04,  3.46016596e-05,  1.75955809e-05,  6.46930951e-06,
        9.04310037e-05, -1.99479382e-05, -1.18206775e-04])

In [46]:
top5_jitter = top5_coords.copy()
top5_jitter['vehicle_location_longitude'] = (top5_jitter['vehicle_location_longitude'] + jitter).round(5)
top5_jitter['vehicle_location_latitude'] = (top5_jitter['vehicle_location_latitude'] + jitter).round(5)
top5_jitter[:10]

,lastLocID,lineColor,delay,vehicle_location_longitude,vehicle_location_latitude
0,8341,Blue,-3831808,-122.66086,45.53008
1,8341,Green,-2364557,-122.66091,45.53004
2,8341,Red,-2309046,-122.66080,45.53015
3,8344,Blue,-4123368,-122.62008,45.53248
4,8344,Green,-2317606,-122.61989,45.53267
5,8344,Red,-2162195,-122.62008,45.53248
6,8345,Blue,-4577134,-122.60165,45.52931
7,8345,Green,-2593475,-122.60155,45.52941
8,8345,Red,-2419728,-122.60150,45.52946
9,8346,Blue,-4986604,-122.57647,45.53282


In [47]:
blueLine_top5 = top5_jitter.loc[top5_jitter['lineColor']=='Blue',:]
greenLine_top5 = top5_jitter.loc[top5_jitter['lineColor']=='Green',:]
redLine_top5 = top5_jitter.loc[top5_jitter['lineColor']=='Red',:]

In [48]:
bdelays = list(map(str, blueLine_top5['delay']))
blueDelays = ['Delay: ' + s for s in bdelays]
bLocID = list(map(str, blueLine_top5['lastLocID']))
blueLocID = ['Stop ID: ' + s for s in bLocID]

blueText = []
for idx in range(len(blueLocID)):
    blueText.append(blueLocID[idx] + ' | ' + blueDelays[idx])
blueText

['Stop ID: 8341 | Delay: -3831808',
 'Stop ID: 8344 | Delay: -4123368',
 'Stop ID: 8345 | Delay: -4577134',
 'Stop ID: 8346 | Delay: -4986604',
 'Stop ID: 8347 | Delay: -6747964']

In [49]:
gdelays = list(map(str, greenLine_top5['delay']))
greenDelays = ['Delay: ' + s for s in gdelays]
gLocID = list(map(str, greenLine_top5['lastLocID']))
greenLocID = ['Stop ID: ' + s for s in gLocID]

greenText = []
for idx in range(len(greenLocID)):
    greenText.append(greenLocID[idx] + ' | ' + greenDelays[idx])
greenText

['Stop ID: 8341 | Delay: -2364557',
 'Stop ID: 8344 | Delay: -2317606',
 'Stop ID: 8345 | Delay: -2593475',
 'Stop ID: 8346 | Delay: -2845667',
 'Stop ID: 8347 | Delay: -3786845']

In [50]:
rdelays = list(map(str, redLine_top5['delay']))
redDelays = ['Delay: ' + s for s in rdelays]
rLocID = list(map(str, redLine_top5['lastLocID']))
redLocID = ['Stop ID: ' + s for s in rLocID]

redText = []
for idx in range(len(redLocID)):
    redText.append(redLocID[idx] + ' | ' + redDelays[idx])
redText

['Stop ID: 8341 | Delay: -2309046',
 'Stop ID: 8344 | Delay: -2162195',
 'Stop ID: 8345 | Delay: -2419728',
 'Stop ID: 8346 | Delay: -2638280',
 'Stop ID: 8347 | Delay: -2696563']

In [51]:
# blue line
trace0 = go.Scattermapbox(
        name='Blue Line',
        lat=blueLine_top5['vehicle_location_latitude'],
        lon=blueLine_top5['vehicle_location_longitude'],
        mode='markers',
        marker=dict(
            size=12,
            color=lineColors['Blue']
        ),
        opacity=1,
        text=blueText
    )

# green line
trace1 = go.Scattermapbox(
        name='Green Line',
        lat=greenLine_top5['vehicle_location_latitude'],
        lon=greenLine_top5['vehicle_location_longitude'],
        mode='markers',
        marker=dict(
            size=12,
            color=lineColors['Green']
        ),
        opacity=1,
        text=greenText
    )

# red line
trace2 = go.Scattermapbox(
        name='Red Line',
        lat=redLine_top5['vehicle_location_latitude'],
        lon=redLine_top5['vehicle_location_longitude'],
        mode='markers',
        marker=dict(
            size=12,
            color=lineColors['Red']
        ),
        opacity=1,
        text=redText,
        hoverinfo='text'
    )

layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=45.53,
            lon=-122.60
        ),
        pitch=0,
        zoom=10.5,
    ),
    margin=dict(l=0,
               r=0,
               t=30,
               b=50),
    legend=dict(font=dict(size=14),
               orientation='h',
               y=1.1)
    
)

fig = dict(data=[trace0, trace1, trace2], layout=layout)
plotly.offline.iplot(fig, filename='top5delays.html')

The top 10 stop IDs with the longest delays are all on the east side of Portland. This is an interesting insight that warrants further investigation.

Finally, the last thing I want to do is plot a heatmap showing the mean delay for each stop.

In [52]:
delays = rail_clean[['lastLocID', 'delay']].groupby('lastLocID').mean().sort_values('delay')
delays[:10]

,delay
lastLocID,
10119,-1000.000000
8350,-262.652274
8355,-259.552642
8351,-241.048450
13450,-232.429548
9836,-226.384004
8349,-224.042050
8354,-223.990069
8356,-220.875552


In [53]:
stop_delays = delays.join(locations)
stop_delays[:10]

,delay,vehicle_location_longitude,vehicle_location_latitude
lastLocID,,,
10119,-1000.000000,-122.682669,45.519415
8350,-262.652274,-122.508260,45.522260
8355,-259.552642,-122.457514,45.514905
8351,-241.048450,-122.492529,45.522129
13450,-232.429548,-122.440305,45.507583
9836,-226.384004,-122.917075,45.530318
8349,-224.042050,-122.528556,45.522418
8354,-223.990069,-122.467001,45.518934
8356,-220.875552,-122.434206,45.505023


The first locID (10119) is a clear outlier at an average of -1000 delay. This stop actually only has two entries in the entire data set and will be excluded.

In [54]:
stop_delays = (stop_delays.iloc[1:, :].reset_index()).round(5)
stop_delays[:10]

,lastLocID,delay,vehicle_location_longitude,vehicle_location_latitude
0,8350,-262.65227,-122.50826,45.52226
1,8355,-259.55264,-122.45751,45.51490
2,8351,-241.04845,-122.49253,45.52213
3,13450,-232.42955,-122.44030,45.50758
4,9836,-226.38400,-122.91708,45.53032
5,8349,-224.04205,-122.52856,45.52242
6,8354,-223.99007,-122.46700,45.51893
7,8356,-220.87555,-122.43421,45.50502
8,8352,-215.67561,-122.48243,45.52204
9,8353,-210.57422,-122.47510,45.52182


In [55]:
locID = list(map(str, stop_delays['lastLocID']))
locIDs = ['Stop ID: ' + s for s in locID]
delay = list(map(str, -stop_delays['delay']))
delays = ['Delay: ' + s for s in delay]

text = []
for idx in range(len(locIDs)):
    text.append(locIDs[idx] + '  |  ' + delays[idx])
text[:10]

['Stop ID: 8350  |  Delay: 262.65227',
 'Stop ID: 8355  |  Delay: 259.55264',
 'Stop ID: 8351  |  Delay: 241.04845',
 'Stop ID: 13450  |  Delay: 232.42955',
 'Stop ID: 9836  |  Delay: 226.384',
 'Stop ID: 8349  |  Delay: 224.04205',
 'Stop ID: 8354  |  Delay: 223.99007',
 'Stop ID: 8356  |  Delay: 220.87555',
 'Stop ID: 8352  |  Delay: 215.67561',
 'Stop ID: 8353  |  Delay: 210.57422']

In [56]:
trace0 = go.Scattermapbox(
        lat=blueLine_top5['vehicle_location_latitude'],
        lon=blueLine_top5['vehicle_location_longitude'],
        mode='markers',
        marker=dict(
            size=18,
            color='rgb(255, 0, 255)',
            colorscale='Portland',
            opacity=.7
        ),
        showlegend=False
    )

trace1 = go.Scattermapbox(
        lat=stop_delays['vehicle_location_latitude'],
        lon=stop_delays['vehicle_location_longitude'],
        mode='markers',
        marker=dict(
            size=9,
            color=-stop_delays['delay'],
            colorscale='Portland',
            showscale=True,
            opacity=1
        ),
        text=text,
        showlegend=False
    )

data=[trace0, trace1]

layout = go.Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=center_lat,
            lon=center_lon
        ),
        pitch=0,
        zoom=9,
    ),
    margin=dict(l=0,
               r=0,
               t=30,
               b=50)
    
)

fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig, filename='TrimetDelayMap.html')

We can size the circles by the color 

## SUPPLEMENTAL

In [57]:
plotly.__version__

'3.1.0'